# Using Classifiers

## 1- Importing libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split as tts

In [2]:
# setting plt layout
plt.ion()

## 2- Importing data

In [3]:
dataset = pd.read_csv('C:/Users/matlp/Desktop/Nova Pasta/KaggleBC/data.csv')

#### Splitting independent and target variables

In [4]:
X = dataset
X = X.drop(X.columns[[1, 32]], axis=1)
y = dataset[['diagnosis']]

#### processing target variable

M = 1,
B = 0

In [5]:
y = y.replace('M', 1).replace('B', 0)


In [6]:
X = X.drop(X.filter(regex='^(?=(radius|perimeter)).*', axis=1).columns, axis=1)


##### Splitting CV and test datasets

Further analysis will be based only on CV

In [7]:
## Separating CV and test datasets
X_cv, X_test, y_cv, y_test = tts(X, y, test_size = 0.1, random_state=1)

## Separating Train and Validation

X_train, X_val, y_train, y_val = tts(X_cv, y_cv, test_size=0.2, random_state=1)

## 5- Feature Scaling

In [9]:
from sklearn.preprocessing import StandardScaler
SC = StandardScaler()
X_cv_N = pd.DataFrame(SC.fit_transform(X_cv), columns=X_cv.columns)
X_test_N = pd.DataFrame(SC.transform(X_test), columns=X_test.columns)

## 4- Fitting classifier

### Random Forest Classifier

#### Without CV (direct model application)

##### Fitting

In [10]:
from sklearn.ensemble import RandomForestClassifier
rtc = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=1)
rtc.fit(X_cv_N, y_cv)

C:\Users\matlp\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=1,
            verbose=0, warm_start=False)

##### Feature importances

In [12]:
importances = rtc.feature_importances_
std = np.std([tree.feature_importances_ for tree in rtc.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X_cv_N.shape[1]):
    print("%d. feature: "%(f+1) + X_cv_N.columns[indices[f]]+ " (%f)" % (importances[indices[f]]))


Feature ranking:
1. feature: area_worst (0.184797)
2. feature: concave points_mean (0.148924)
3. feature: concavity_mean (0.130332)
4. feature: concave points_worst (0.091346)
5. feature: concavity_worst (0.077628)
6. feature: area_se (0.071919)
7. feature: area_mean (0.051836)
8. feature: smoothness_worst (0.048815)
9. feature: compactness_mean (0.047531)
10. feature: concavity_se (0.023527)
11. feature: texture_worst (0.017272)
12. feature: texture_mean (0.016815)
13. feature: concave points_se (0.012851)
14. feature: fractal_dimension_mean (0.010979)
15. feature: symmetry_worst (0.010203)
16. feature: id (0.009394)
17. feature: compactness_se (0.008925)
18. feature: symmetry_se (0.008155)
19. feature: symmetry_mean (0.006572)
20. feature: fractal_dimension_worst (0.006521)
21. feature: texture_se (0.004582)
22. feature: fractal_dimension_se (0.004181)
23. feature: compactness_worst (0.003307)
24. feature: smoothness_mean (0.003161)
25. feature: smoothness_se (0.000424)


That's close to what we expected after the feature exploration

##### Prediction

In [13]:
y_rtc = rtc.predict(X_test_N)

##### Metrics

In [14]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [15]:
print(classification_report(y_test,y_rtc))

             precision    recall  f1-score   support

          0       0.94      1.00      0.97        34
          1       1.00      0.91      0.95        23

avg / total       0.97      0.96      0.96        57



In [16]:
confusion_matrix(y_test,y_rtc)

array([[34,  0],
       [ 2, 21]])

In [17]:
accuracy_score(y_test,y_rtc)

0.9649122807017544